In [6]:
import cv2
import os
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
from pygame import mixer

In [7]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
left_eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_lefteye_2splits.xml')
right_eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_righteye_2splits.xml')
model =load_model('models/model.h5')

In [21]:
mixer.init()
sound = mixer.Sound('alarm.wav')
path = os.getcwd()
font = cv2.FONT_HERSHEY_COMPLEX_SMALL
cap = cv2.VideoCapture(0)
score = 0
while True:
    ret, frame = cap.read()
    height,width = frame.shape[0:2]
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces= face_cascade.detectMultiScale(gray, scaleFactor= 1.2, minNeighbors=3)
    left_eye= left_eye_cascade.detectMultiScale(gray, scaleFactor= 1.1, minNeighbors=1)
    right_eye= right_eye_cascade.detectMultiScale(gray, scaleFactor= 1.1, minNeighbors=1)
    
    cv2.rectangle(frame, (0,height-50),(200,height),(0,0,0),thickness=cv2.FILLED)
    
    for (x,y,w,h) in faces:
        cv2.rectangle(frame,pt1=(x,y),pt2=(x+w,y+h), color= (255,255,255), thickness=1 )
        
    for (ex,ey,ew,eh) in right_eye:
        cv2.rectangle(frame,pt1=(ex,ey),pt2=(ex+ew,ey+eh), color= (255,255,255), thickness=1 )
        
        # preprocessing steps
        reye= frame[ey:ey+eh,ex:ex+ew]
        reye= cv2.resize(reye,(80,80))
        reye= reye/255
        reye= reye.reshape(80,80,3)
        reye= np.expand_dims(reye,axis=0)
        # preprocessing is done now model prediction
        rprediction = model.predict(reye)
        
        for (ex,ey,ew,eh) in left_eye:
            cv2.rectangle(frame,pt1=(ex,ey),pt2=(ex+ew,ey+eh), color= (255,255,255), thickness=1 )

                # preprocessing steps
            leye= frame[ey:ey+eh,ex:ex+ew]
            leye= cv2.resize(leye,(80,80))
            leye= leye/255
            leye= leye.reshape(80,80,3)
            leye= np.expand_dims(leye,axis=0)
                # preprocessing is done now model prediction
            lprediction = model.predict(leye)

                # if eyes are closed
            if rprediction[0][0]>0.80 and lprediction[0][0]>0.80:
                cv2.putText(frame,"Closed",(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
                cv2.putText(frame,'Score:'+str(score),(100,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
                score=score+1
                if(score>6):
                    cv2.imwrite(os.path.join(path,'image.jpg'),frame)
                    try:
                        sound.play()
                    except:
                        pass

            else:
                cv2.putText(frame,'Open',(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)  
                cv2.putText(frame,'Score:'+str(score),(100,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
                score = score-1
                if (score<0):
                    score=0


    cv2.imshow('frame',frame)
    if cv2.waitKey(33) & 0xFF==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 68ms/step
